# Task 1: Binary Representations

Optional part: Calculate how many bits set in code, bits set is number of 1s

## 1. Implement Bitwise Left Rotation (`rotl`)

In [9]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotl(x, n=1):
    n %= 32  # Ensures n is within 0-31
    return ((x << n) & MASK_32) | ((x & MASK_32) >> (32 - n))

### Tests

In [28]:
if __name__ == "__main__":
    test_val = 0x12345678
    
    result_4 = rotl(test_val, 4)
    result_0 = rotl(test_val, 0)

    # Function to format binary output with leading zeros
    def to_bin_str(val):
        return bin(val)[2:].zfill(32)  # Ensures it's always 32 bits long

    print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
    print(f"rotl(4):    0x{result_4:X} ({to_bin_str(result_4)})")
    print(f"rotl(0):    0x{result_0:X} ({to_bin_str(result_0)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotl(4):    0x23456781 (00100011010001010110011110000001)
rotl(0):    0x12345678 (00010010001101000101011001111000)


## 2. Implement Bitwise Right Rotation (`rotr`)


In [14]:
MASK_32 = 0xFFFFFFFF  # 32-bit mask to prevent overflow

def rotr(x, n=1):
    
    n %= 32  # Ensure n is within 0-31
    return ((x >> n) & MASK_32) | ((x << (32 - n)) & MASK_32)


### Tests

In [16]:
if __name__ == "__main__":
    test_val = 0x12345678 
    
    result_4 = rotr(test_val, 4)
    result_1 = rotr(test_val, 1)

    def to_bin_str(val):
        return bin(val)[2:].zfill(32)

    print(f"Original:   0x{test_val:X} ({to_bin_str(test_val)})")
    print(f"rotr(4):    0x{result_4:X} ({to_bin_str(result_4)})")
    print(f"rotr(1):    0x{result_1:X} ({to_bin_str(result_1)})")

Original:   0x12345678 (00010010001101000101011001111000)
rotr(4):    0x81234567 (10000001001000110100010101100111)
rotr(1):    0x91A2B3C (00001001000110100010101100111100)


## 3. Implement `ch(x, y, z)` Function


In [1]:
# chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
def ch(x, y, z):
    return (x & y) ^ (~x & z)


### Tests

In [5]:
# x is diferent from both y and z
x_val = 0b1010 
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to y and diferent from z
x_val = 0b0000
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

# x is equal to z and diferent from y
x_val = 0b1111
y_val = 0b1100
z_val = 0b1111
result = ch(x_val, y_val, z_val)
# prints result in binary, with at least 4 bits (padded with zeros if necessary).
print(f"ch(0b1010, 0b1100, 0b1111) = 0b{result:04b}")

ch(0b1010, 0b1100, 0b1111) = 0b1101
ch(0b1010, 0b1100, 0b1111) = 0b1111
ch(0b1010, 0b1100, 0b1111) = 0b1100


# Task 2

# Task 3

# Task 4

# Task 5

# Task 6

# Task 7

# Task 8